In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import converter, instrument, note, chord, stream
from keras import backend as K
from keras import Input
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Permute
from keras.layers import RepeatVector
from keras.layers import Multiply
from keras.layers import Lambda
from keras.optimizers import RMSprop
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [3]:
# dataset_name = 'jazz'
# filename = 'jazz_file'
# file = "midi/jazz2/{}.mid".format(dataset_name, filename)

notes = []
durations = []
for file in glob.glob("midi/jazz2/*.mid"):
    try:
        original_score = converter.parse(file).chordify()
    except Exception as inst:
        print(inst)
        pass        
    
    for element in original_score.flat:
        if isinstance(element, chord.Chord):
            notes.append('.'.join(n.nameWithOctave for n in element.pitches))
            durations.append(element.duration.quarterLength)
            
        if isinstance(element, note.Note):
            if element.isRest:
                notes.append(str(element.name))
                durations.append(element.duration.quarterLength)
                
            else:
                notes.append(str(element.nameWithOctave))
                durations.append(element.duration.quarterLength)
                
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
duration_names = sorted(set(item for item in durations))
duration_to_int = dict((note, number) for number, note in enumerate(duration_names))
    
print(pitchnames)
print(note_to_int)
print(duration_names)
print(duration_to_int)
print("\n")
#     print(notes)
#     print(durations)
#     print("\n")

        
#     print(original_score.show('text'))

cannot handle ticks per frame: 144
cannot handle ticks per frame: 144
cannot handle ticks per frame: 144
['A2', 'A2.B-2', 'A2.B2', 'A2.B2.E-3', 'A2.B3', 'A2.C#3', 'A2.C3', 'A2.D3', 'A2.E-3', 'A2.E3', 'A2.F#3', 'A2.F3', 'A3', 'A3.A4', 'A3.B-3', 'A3.B-3.B3', 'A3.B-3.B3.C4', 'A3.B-3.C#4', 'A3.B-3.C4', 'A3.B-3.D4', 'A3.B3', 'A3.B3.C#4', 'A3.B3.C#4.D4', 'A3.B3.C4', 'A3.B3.C4.C#4', 'A3.B3.D4', 'A3.B3.D4.E4', 'A3.B3.E-4.F#4', 'A3.B3.E-4.F4', 'A3.B3.E4', 'A3.B4', 'A3.C#4', 'A3.C#4.D4', 'A3.C#4.E-4', 'A3.C#4.F4', 'A3.C#5', 'A3.C4', 'A3.C4.C#4', 'A3.C4.D4.E-4', 'A3.C4.D4.G4', 'A3.C4.E-4', 'A3.C4.E4', 'A3.D4', 'A3.D4.A4', 'A3.D4.F#4', 'A3.E-4', 'A3.E-4.E4', 'A3.E4', 'A3.E4.A4', 'A3.E4.F4', 'A3.F#4', 'A3.F#4.A4', 'A3.F4', 'A3.G4', 'A4', 'A4.B-4', 'A4.B-4.B4', 'A4.B-4.C#5', 'A4.B-4.C5', 'A4.B-4.C5.D5', 'A4.B-4.D5', 'A4.B-4.E-5', 'A4.B-4.G5', 'A4.B4', 'A4.B4.C#5', 'A4.B4.C5', 'A4.B4.D5', 'A4.B4.E5', 'A4.C#5', 'A4.C#5.E5', 'A4.C5', 'A4.C5.C#5', 'A4.C5.D5', 'A4.C5.D5.E5', 'A4.C5.E5', 'A4.D5', 'A4.E-5'

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#translates original list with dicts
notes_trans = [note_to_int[note] for note in notes]
durations_trans = [duration_to_int[duration] for duration in durations]

swift_number = 32 #length of one-hot vector
step = 31 #one hot encoding overlap 


#notes_chunks = [notes_trans[base:base+swift_number] for base in range(0,len(notes_trans), step)]
notes_chunks = []
notes_output = []
for base in range(0,len(notes_trans), step):
    seq = notes_trans[base:base+swift_number]
    if len(seq) == 32:
        notes_chunks.append(seq)
        el = seq[-1]
        notes_output.append(el)
#     else:
#         notes_trans = [elem for elem in notes_trans if elem not in seq]
#notes_output = []
# for seq in notes_chunks:
#     el = seq[-1]
#     notes_output.append(el)
#     if len(seq) != 32:
#         notes_chunks.remove(seq)
#         notes_output.remove(el)
    
        
#durations_chunks = [durations_trans[base:base+swift_number] for base in range(0,len(durations_trans), step)]
durations_chunks = []
durations_output = []
for base in range(0,len(durations_trans), step):
    seq = notes_trans[base:base+swift_number]
    if len(seq) == 32:
        durations_chunks.append(seq)
        el = seq[-1]
        durations_output.append(el)
#     else:
#         durations_trans = [elem for elem in notes_trans if elem not in seq]
#durations_output = []
# for seq in durations_chunks:
#     el = seq[-1]
#     durations_output.append(el)
#     if len(seq) != 32:
#         durations_chunks.remove(seq)
#         durations_output.remove(el)


def onehotencode(chunks_list):
    enc_list = []
    for data in chunks_list:
        values = np.array(data)
        #values = data
        #print(values)
        #binary onehot encoding
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = values.reshape(len(values),1)
        onehot_encoded = onehot_encoder.fit(integer_encoded)
        onehot_encoded = onehot_encoder.transform(integer_encoded)
#         onthot_encoded = to_categorical(values)
        enc_list.append(onehot_encoded)
        break
        #print(onehot_encoded)
    return enc_list
        #inverted = [ label_encoder.inverse_transform([np.argmax(onehot_encoded[i, :])]) for i in range(len(onehot_encoded))]

def onehotdecode(enc_list):
    return [ label_encoder.inverse_transform([np.argmax(onehot_encoded[i, :])]) for i in range(len(onehot_encoded))]

print(len(set(notes)))
print(len(set(durations)))
#print(type(notes_chunks[0]))
#print(type(durations_chunks[0]))
#print(notes_chunks)
print(durations_chunks)

notes_enc = onehotencode(notes_chunks)
durations_enc = onehotencode(durations_chunks)



1204
112
[[155, 260, 54, 55, 155, 678, 776, 367, 577, 367, 260, 155, 841, 1051, 939, 939, 939, 939, 1146, 531, 939, 562, 531, 321, 1146, 531, 432, 414, 1051, 531, 414, 1051], [1051, 54, 939, 455, 414, 12, 1099, 108, 150, 841, 732, 414, 321, 531, 531, 550, 732, 939, 940, 54, 776, 155, 260, 54, 1146, 155, 414, 321, 531, 108, 414, 155], [155, 54, 1052, 1051, 54, 1147, 1146, 939, 321, 155, 464, 577, 155, 169, 464, 54, 155, 155, 169, 464, 54, 1146, 977, 939, 1146, 54, 155, 169, 464, 367, 577, 464], [464, 155, 1051, 1146, 321, 841, 1146, 1051, 939, 661, 639, 577, 367, 260, 464, 155, 260, 464, 1171, 1146, 1146, 732, 1146, 841, 531, 108, 732, 939, 1146, 732, 939, 1146], [1146, 54, 155, 464, 1146, 1051, 841, 531, 108, 321, 531, 211, 260, 269, 464, 884, 464, 54, 1146, 155, 176, 577, 414, 54, 1146, 776, 1189, 884, 1083, 776, 577, 464], [464, 54, 155, 54, 939, 776, 1189, 776, 577, 473, 464, 776, 782, 983, 464, 54, 155, 1146, 939, 531, 532, 54, 464, 155, 155, 54, 939, 1146, 54, 55, 155, 464], [464,

In [22]:
# def create_network(notes_in, n_notes, durations_in, n_durations):
#model parameters
batch_size = 64
embedding_dim = 100 #256
vocab_notes = len(set(notes)) + 1
vocab_durations = len(set(durations)) + 1

notes_input = np.reshape(notes_chunks, (len(notes_chunks), 32, 1))
#notes_input = notes_input / float(vocab_notes)
durations_input = np.reshape(durations_chunks, (len(durations_chunks), 32, 1))
#notes_input = notes_input / float(vocab_durations)

notes_output = np_utils.to_categorical(notes_output)
durations_output = np_utils.to_categorical(durations_output)

notes_in = Input(shape = (None,))
durations_in = Input(shape = (None,))

n_notes = len(set(notes)) + 1
n_durations = len(set(durations)) + 1

############################################
rnn_units = 256
x1 = Embedding(n_notes, embedding_dim)(notes_in)
x2 = Embedding(n_durations, embedding_dim)(durations_in)

x = Concatenate()([x1,x2])

x = LSTM(rnn_units, return_sequences=True)(x)
x = LSTM(rnn_units, return_sequences=True)(x)

e = Dense(1, activation='tanh')(x)
e = Reshape([-1])(e)

alpha = Activation('softmax')(e)

c = Permute([2,1])(RepeatVector(rnn_units)(alpha))
c = Multiply()([x,c])
c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)

notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
durations_out = Dense(n_durations, activation = 'softmax', name='duration')(c)

model = Model([notes_in, durations_in], [notes_out, durations_out])

att_model = Model([notes_in, durations_in], alpha)

opti = RMSprop(learning_rate = 0.001)
model.compile(loss=['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opti)
    
#     return model

checkpoint = ModelCheckpoint(
    "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
) 

callbacks_list = [checkpoint]

model.fit(x=[notes_input, durations_input],y=[notes_output, durations_output], epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

InvalidArgumentError:  indices[31,8] = 321 is not in [0, 113)
	 [[node model_24/embedding_27/embedding_lookup (defined at /home/dcuglian/.local/lib/python3.7/site-packages/keras/layers/embeddings.py:184) ]] [Op:__inference_train_function_51831]

Errors may have originated from an input operation.
Input Source operations connected to node model_24/embedding_27/embedding_lookup:
 model_24/embedding_27/Cast (defined at /home/dcuglian/.local/lib/python3.7/site-packages/keras/layers/embeddings.py:183)	
 model_24/embedding_27/embedding_lookup/49303 (defined at /opt/conda/lib/python3.7/contextlib.py:112)

Function call stack:
train_function


In [9]:
# def train_network():
#     notes_in = Input(shape = (None,))
#     durations_in = Input(shape = (None,))
#     n_notes = len(notes_enc)
#     n_durations = len(durations_enc)

#     print(notes_enc)
#     print(durations_enc)
#     model = create_network(notes_enc, len(set(notes)), durations_enc, len(set(durations)))
    
#     checkpoint = ModelCheckpoint(
#         "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
#         monitor='loss',
#         verbose=0,
#         save_best_only=True,
#         mode='min'
#     ) 
    
#     callbacks_list = [checkpoint]

#     # line below still not working
#     #model.fit(x=[network_input, durations_in],y=[notes_out, durations_out],epochs=50, callbacks=callbacks_list)
#     model.fit(x=[network_input_n, network_input_d],y=[network_output_n, network_output_d],epochs=50, callbacks=callbacks_list)
# train_network()
# #print("DONE WITH TRAINING")

# #print("NEW CELL")

In [6]:
# def create_network(notes_in, n_notes, durations_in, n_durations):
#model parameters
batch_size = 64
embedding_dim = 32 #256
vocab_notes = len(set(notes)) 
vocab_durations = len(set(durations)) 

notes_input = np.reshape(notes_chunks, (len(notes_chunks), 32, 1))
#notes_input = notes_input / float(vocab_notes)
durations_input = np.reshape(durations_chunks, (len(durations_chunks), 32, 1))
#notes_input = notes_input / float(vocab_durations)

notes_output = np_utils.to_categorical(notes_output)
durations_output = np_utils.to_categorical(durations_output)

#notes_in = Input(shape = (None,))
#durations_in = Input(shape = (None,))

n_notes = len(set(notes))
n_durations = len(set(durations))

############################################

rnn_units = 128

model = Sequential()
model.add(LSTM(
    512,
    input_shape=(notes_input.shape[1], notes_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))

''' Your Code Here '''
#LSTM layer
model.add(LSTM(
    512,
    recurrent_dropout=0.3
))
#batch normalization layer
#model.add(BatchNormalization())
#dropout layer
model.add(Dropout(0.3))
#fully connected layer
model.add(Dense(256))
#relu
model.add(Activation('relu'))
#batch normalization layer
model.add(BatchNorm())
#dropout layer
model.add(Dropout(0.3))
#fully connected layer
model.add(Dense(n_notes)) #fix this one
#temperature
model.add(Lambda(lambda x: x/ .6))
#softmax
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer='adam')

#return model

checkpoint = ModelCheckpoint(
    "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
) 

callbacks_list = [checkpoint]

    # Your line of code here
model.fit(notes_input, notes_output, epochs=50, callbacks=callbacks_list)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

ValueError: Shapes (None, 1203, 2) and (None, 1204) are incompatible